In [1]:
import pecnet as pc
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

from models import Model, FinalModel

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import tensorflow as tf
import os 
import random
#This is for getting the same results. 
def reset_random_seeds(seed):
   os.environ['PYTHONHASHSEED']=str(seed)
   tf.random.set_seed(seed)
   np.random.seed(seed)
   random.seed(seed)
reset_random_seeds(42)

In [ ]:
input_data = pd.read_csv('close_is.csv').to_numpy().flatten()

WINDOW_SIZE = 4
SPLIT_INDEX = 3546 #3538 - 66 split, like Ajla's work
#We are starting from 4 because we are constructing a rolling mean window from the same dataset. We want to align the split indexes for every network. 
ROLLING_WINDOW_SIZE = 5
START = ROLLING_WINDOW_SIZE - 1
ERROR_WINDOW_SIZE = 4


#Average
roll = pc.rolling_mean(input_data, ROLLING_WINDOW_SIZE) #yy5
X_train, X_test, y_train, y_test, mean = pc.prepare_and_split_data(roll, WINDOW_SIZE, SPLIT_INDEX, start=START)

#Succesive
X_train_2, X_test_2, y_train_2, y_test_2, mean_2 = pc.prepare_and_split_data(input_data, WINDOW_SIZE, SPLIT_INDEX, start=START)

In [ ]:
#We are using raw values for y. Not the average ones.
#Also we will use mean of windowized input_data to denormalize
y_test = y_test_2
y_train = y_train_2
mean = mean_2

In [ ]:
first_model = Model(X_train.shape, y_train.shape)
first_model.set_model_info("First Model")
first_model.set_fit_args()
first_model.init_model()
# first_model.model.summary()
hist = first_model.fit_model(X_train, y_train, verbose=False)

In [ ]:
#Predict for first model
predict_train = first_model.model.predict(X_train).flatten()
predict_test = first_model.model.predict(X_test).flatten()

#Calculate First Model's errors
error_train = predict_train - y_train
error_test = predict_test - y_test

In [ ]:
#Correlation Matrix
np.corrcoef(error_train, y_train_2)

In [ ]:
#Second model, use error of the first one
second_model = Model(X_train_2.shape, error_train.shape)
second_model.set_model_info("Second Model")
second_model.set_fit_args()
second_model.init_model()
# second_model.model.summary()
hist_2 = second_model.fit_model(X_train_2, error_train, verbose=False)

In [ ]:
#Predict for second model
predict_train_2 = second_model.model.predict(X_train_2).flatten()
predict_test_2 = second_model.model.predict(X_test_2).flatten()
#Calculate compensated errors
error_train_compensated = (predict_train + mean[:X_train.shape[0]]) - predict_train_2
error_test_compensated = (predict_test + mean[X_train.shape[0]:]) - predict_test_2
#Calculate error for second model.
error_train_2 = error_train_compensated - (y_train + mean[:X_train.shape[0]])
error_test_2 = error_test_compensated - (y_test + mean[X_train.shape[0]:])

In [ ]:
#Merge errors to create an error timeseries. 
error_all = np.concatenate([[0,0,0,0],error_train_2, error_test_2])
#This is for aligning error data and other data.
#Set this one to zero if you are adding zero padding to error_all. If not, make it same as ERROR_WINDOW_SIZE
ERROR_ALIGNMENT = 0

#For dividing errors from the same index.
ERROR_SPLIT_INDEX = SPLIT_INDEX - ERROR_ALIGNMENT - START

#Divide from the same point. 
X_train_error, X_test_error, y_train_error, y_test_error, mean_error = pc.prepare_and_split_error_data(error_all, ERROR_WINDOW_SIZE, ERROR_SPLIT_INDEX, normalize=True, fill=False)

In [ ]:
#Third model, Error network
error_model = Model(X_train_error.shape, y_train_error.shape)
error_model.set_model_info("Error Model")
error_model.set_fit_args()
error_model.init_model()
# error_model.model.summary()
hist_3 = error_model.fit_model(X_train_error, y_train_error, verbose=False)

In [ ]:
#Predict for error Model
predict_train_3 = error_model.model.predict(X_train_error).flatten()
predict_test_3 = error_model.model.predict(X_test_error).flatten()

#Calculate compensated errors.
error_train_compensated_2 = error_train_compensated[ERROR_ALIGNMENT:] - (predict_train_3 + mean_error[:X_train_error.shape[0]])
error_test_compensated_2 = error_test_compensated - (predict_test_3 + mean_error[X_train_error.shape[0]:])

#Calculate error for error model.
error_train_3 = (predict_train_3 + mean_error[:X_train_error.shape[0]])
error_test_3 = (predict_test_3 + mean_error[X_train_error.shape[0]:])

In [ ]:
#Create Final Network IO
X_train_final = np.column_stack((predict_train[ERROR_ALIGNMENT:], predict_train_2[ERROR_ALIGNMENT:], error_train_3)) 
X_test_final = np.column_stack((predict_test, predict_test_2, error_test_3))

y_train_final = y_train[ERROR_ALIGNMENT:]
y_test_final = y_test

In [ ]:
#Final model
final_model = FinalModel(X_train_final.shape, y_train_final.shape)
final_model.set_model_info("Final Model")
final_model.set_fit_args()
final_model.init_model()
# final_model.model.summary()
hist_final = final_model.fit_model(X_train_final, y_train_final, verbose=False)

In [ ]:
#Predict the values
predict_train_final = final_model.predict(X_train_final).flatten()
predict_test_final = final_model.predict(X_test_final).flatten()
#Add mean
predict_train_final = predict_train_final + mean_2[ERROR_ALIGNMENT:X_train.shape[0]]
predict_test_final = predict_test_final + mean_2[X_train.shape[0]:]

In [ ]:
fig, ((ax0,ax1),(ax2,ax3)) = plt.subplots(2,2, figsize=(10,10))
ax0.plot(hist.history["loss"])
ax0.set_title("First Network (X=Average)")

ax1.plot(hist_2.history["loss"])
ax1.set_title("Second Network (X=Raw)")

ax2.plot(hist_3.history["loss"])
ax2.set_title("Error Network")

ax3.plot(hist_final.history["loss"])
ax3.set_title("Final Network")

In [ ]:
plt.figure(figsize=(10,5))
plt.plot((y_test_2 + mean[y_train_2.shape[0]:])[:-1], c='b', label="Real")
plt.plot(predict_test_final[:-1], c='r', label="Predicted")
plt.legend()
plt.show()

In [ ]:
mean_squared_error((y_test_2 + mean[y_train_2.shape[0]:])[:-1], predict_test_final[:-1], squared=False)

In [ ]:
predict_test_final[-1]